**2. Preprocess**

1. Mygene annotation
(Optional : 1-1. Annotation sanity check)
2. QC (Basic mt/rps/hb HVG subset) + Basic downstream pre-scVI
3. Setup SCVI model on HVGs

-----------------------------------------
*Logs*

20251020 Create RERUN_PREPROCESS : Unpack .tar.gz to read and concatenate.

20251020 Revise ANNOTATE_MYGENE : Fix logic flow & memory efficiency. Drop sanity check : ~30% of dropout is allowed

20251020 Revise SCVI_SETUP : Fix learning parameters. Reduce keys to prevent overfitting

20251025 Revise HVG SELECTION : Subset hvgs first before scVI for robust results

20251102 Creat QC_SCHEME : Remove outlier cells/genes with 3MAD rule
(filter cell/gene -> MT/RPS/HB/total counts/pct counts masking)

20251108 FIx QC_SCHEME : Manual filter_cells/genes to save memory

20251216 Refactoring : fixed code flow and cleared paths

In [ ]:
# only for Colab

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/repos/Epilepsy_Microglia
%pip install -q -r requirements.txt

In [ ]:
# project name
project = "Epilepsy_Microglia"
FNAME = "thrupp"
FCODE = "GSE153807"

# environment setting
from env_utils import detect_env, get_paths
from sc_utils import sc_unpack_tar

env = detect_env()
paths = get_paths(project)

# ML Libraries
import torch

# Single Cell Libraries
import scvi
import scanpy as sc
import anndata as ad
from scar import model, setup_anndata

# Data Processing and Plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import igraph
import leidenalg
import random

# File grab
import os
import tempfile
import pooch
import shutil, subprocess, glob
import gzip

# Trivia 
from datetime import date
TODAY = date.today()
print("Today's date: ", TODAY)

# Version & sanity check
print(torch.__version__)
print(scvi.__version__)
print(torch.cuda.is_available())
import warnings
warnings.simplefilter("ignore")

# Random key
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
scvi.settings.seed = SEED

# Pathways and setting
sc.set_figure_params(dpi_save=300, frameon=False)
sc.settings.figdir = paths["PLOTS"]

BASE = paths["BASE"]
FLABEL = f"{FCODE}_{TODAY}"

RAW_DIR = paths["RAW"] / FNAME
PROCESSED_DIR = paths["PROCESSED"] / FNAME

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

Seed set to 42


Today's date:  2025-12-16
2.9.1+cu128
1.3.3
False


In [ ]:
sc_unpack_tar(RAW_DIR, FCODE)

In [ ]:
# @title
FNAME = "GSE153807"
FLABEL = "thrupp_20251106"

RAW_DIR = "/content/drive/MyDrive/datas/epilepsy_microglia/raw/GSE153807_raw"
OUT_DIR = "/content/drive/MyDrive/datas/epilepsy_microglia/processed/GSE153807"

os.makedirs(OUT_DIR, exist_ok=True)

adatas = {}

from pathlib import Path

for file in glob.glob(os.path.join(RAW_DIR, "*.h5ad")):
    a = sc.read_h5ad(file)
    # fix orientation
    X = a.X.T
    obs = pd.DataFrame(index=a.var_names)
    var = pd.DataFrame(index=a.obs_names)
    a = ad.AnnData(X, obs=obs, var=var)
    a.var_names_make_unique()

    # assign sample id
    sample_id = Path(file).stem
    a.obs["sample"] = sample_id

    adatas[sample_id] = a
    print(f"Loaded {sample_id} with shape {a.shape}")

if adatas:
    adata = sc.concat(
        adatas.values(),
        label="sample",
        keys=adatas.keys(),
        merge="first"
    )

    adata.obs_names_make_unique()
    print(adata.obs["sample"].value_counts())
    adata.write(os.path.join(OUT_DIR, f"{FLABEL}.h5ad"))

In [ ]:
# not for github
# Download

adata = sc.read_h5ad(os.path.join(OUT_DIR, f"{FLABEL}.h5ad"))
adata

In [ ]:
adata.write(os.path.join(OUT_DIR, f"{FLABEL}.h5ad"))

In [ ]:
# not for github
# Upload (for the first run)

adata_qc.write(os.path.join(OUT_DIR, "kumar_qc_20251101v3.h5ad"))
adata_hvg.write(os.path.join(OUT_DIR, "kumar_hvg_20251101v3.h5ad"))

1. Annotate_Mygene : Map ENSG to HGNC symbols, Dump unmapped genes

1-1. Optional Sanity Check : Statistical testing to make sure unmapped genes are worth dumping

In [ ]:
# @title
## 1. ANNOTATE_MYGENE
# R : 20251020

# Annotate ENSG with HGNC symbols

!pip install mygene
import mygene

mg = mygene.MyGeneInfo()

# 현재 gene_ids는 ensg .. .1, .2 이런 데이터로 되어 있음
# 해당 gene_ids에서 소수점을 제거하고 ensembl_id라는 새로운 column으로 저장하기

adata.var['ensembl_id'] = adata.var['gene_ids'].astype(str).str.split('.').str[0]
genes = adata.var['ensembl_id'].dropna().unique().tolist()

res_list = []
for chunk in [genes[i:i+1000] for i in range(0, len(genes), 1000)]:
    res_part = mg.querymany(chunk, scopes="ensembl.gene", fields="symbol", species="human")
    res_list.extend(res_part)
df = pd.DataFrame(res_list)
df = df[~df['symbol'].isna()].drop_duplicates(subset='query').rename(columns={'query':'ensembl_id'})

# Merge
adata.var['gene_symbol'] = adata.var['ensembl_id'].map(df.set_index('ensembl_id')['symbol'])

# Drop NaNs
adata = adata[:, ~adata.var['gene_symbol'].isna()].copy()
adata.var_names_make_unique()
adata.var

In [ ]:
# @title
# 1-1 (Optional) : Sanity check of mapped subset

adata_mapped = adata[:, adata.var['status']=='mapped'].copy()
sc.pp.highly_variable_genes(
    adata_mapped,
    flavor='seurat_v3',
    n_top_genes=3000
)
adata.var['mapped_highly_variable'] = False
adata.var.loc[adata_mapped.var_names[adata_mapped.var['highly_variable']], 'mapped_highly_variable'] = True

# Fisher's exact test - 1. Checking if 'mapped' and 'highly_variable' are independent
!pip install scipy
from scipy.stats import fisher_exact

table = pd.crosstab(
    adata.var['status']=='mapped',
    adata.var['highly_variable']
)

odds, p = fisher_exact(table)
print(table)
print(f"Odds ratio : {odds:.2f}, p-value : {p:2e}")

# Check HVG overlap
hvg_all = set(adata.var_names[adata.var['highly_variable']])
hvg_mapped = set(adata.var_names[adata.var['mapped_highly_variable']])

overlap = len(hvg_all & hvg_mapped)
print(f"HVG overlap: {overlap} / {len(hvg_all)} = {overlap/len(hvg_all):.1%}")

In [ ]:
## 2. QC
adata.layers["counts"] = adata.X.copy()
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True)

sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo", "pct_counts_hb"],
    jitter = 0.4,
    multi_panel=True,
    save = "thrupp_qc_20251107.png"
)
sc.pl.scatter(
    adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt",
    save = "thrupp_qc_20251107.png"
)

# QC scheme 20251102
MT_THRESHOLD = 20
RIBO_THRESHOLD = 80
HB_THRESHOLD = 10
# n_genes_by_counts : log1p transform + 3MAD filter
# total_counts : log1p transform + 3MAD filter
# min_genes 100, min_cells 3


In [ ]:
# run manual filtering instead of memory-heavy sc.pp.filter_genes/cells

adata.obs["n_genes"] = np.ravel((adata.X > 0).sum(axis=1))
adata.var["n_cells"] = np.ravel((adata.X > 0).sum(axis=0))

adata = adata[adata.obs["n_genes"] > 200, :]
adata = adata[:, adata.var["n_cells"] > 3]

In [ ]:
# QC scheme 20251102
MT_THRESHOLD = 20
RIBO_THRESHOLD = 80
HB_THRESHOLD = 10
# n_genes_by_counts : log1p transform + 3MAD filter
# total_counts : log1p transform + 3MAD filter
# min_genes 100, min_cells 3

# 1. Compute log-transformed values
adata.obs["log_n_genes"] = np.log1p(adata.obs["n_genes_by_counts"])
adata.obs["log_total_counts"] = np.log1p(adata.obs["total_counts"])

# 2. Define a helper function for MAD filtering
def mad_filter(series, n_mad=3):
    x = np.log1p(series)
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    lower = med - n_mad * mad
    upper = med + n_mad * mad
    return (x > lower) & (x < upper)

# 3. Build QC mask
mask_genes = mad_filter(adata.obs["n_genes_by_counts"], n_mad=3)
mask_counts = mad_filter(adata.obs["total_counts"], n_mad=3)
mask_mt = adata.obs["pct_counts_mt"] < MT_THRESHOLD
mask_ribo = adata.obs["pct_counts_ribo"] < RIBO_THRESHOLD
mask_hb = adata.obs["pct_counts_hb"] < HB_THRESHOLD

QC_MASK = mask_genes & mask_counts & mask_mt & mask_ribo & mask_hb
print(f"Cells kept: {QC_MASK.sum()} / {adata.n_obs}")

# 4. Plot before filtering
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(np.log1p(adata.obs["n_genes_by_counts"]), bins=100)
axes[0].set_title("log(n_genes_by_counts) - before")
axes[1].hist(np.log1p(adata.obs["total_counts"]), bins=100)
axes[1].set_title("log(total_counts) - before")
plt.show()

# 5. Apply mask
adata_qc = adata[QC_MASK, :].copy()

# 6. Plot after filtering
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(np.log1p(adata_qc.obs["n_genes_by_counts"]), bins=100)
axes[0].set_title("log(n_genes_by_counts) - after")
axes[1].hist(np.log1p(adata_qc.obs["total_counts"]), bins=100)
axes[1].set_title("log(total_counts) - after")
plt.show()

In [ ]:
# 3. Model setup
# R : 20251025

scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="sample_id",
    categorical_covariate_keys=["region"],
)
model = scvi.model.SCVI(adata, n_layers=2, n_latent=20)
model.train()
model_dir = os.path.join(OUT_DIR, "model_v3")
os.makedirs(model_dir, exist_ok=True)
model.save(model_dir, save_anndata=False, overwrite=True)

This is the end of the notebook

In [ ]:
# @title
## Dumped 20251020 : too many keys erase biology.
# 2. Model setup
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="sample_id",
    categorical_covariate_keys=[
        "patient_id", "sample_id", "dataset", "sex", "dx", "dx_subtype",
        "region", "hemisphere", "procedure", "protocol"
    ],
    continuous_covariate_keys=[
        "age"
    ]
)

In [ ]:
# @title
# Dumped 20251020 : too many logical flaws
# 2. Annotate ENSG with HGNC symbols

!pip install mygene
import mygene

mg = mygene.MyGeneInfo()

genes = list(adata.var["gene_ids"])
res = mg.querymany(
    genes,
    scopes="ensembl.gene",
    fields="symbol",
    species="human",
    returnall=True
)

# Convert MyGene results into a DataFrame
df = pd.DataFrame(res.get('out', []))

# Extract clean Ensembl IDs (remove version suffixes)
df['ensembl_id'] = df['query'].str.split('.').str[0]

# Keep only rows with a valid symbol
df = df[~df['symbol'].isna()].drop_duplicates(subset='ensembl_id')

# Join by Ensembl ID
adata.var = adata.var.join(df.set_index('ensembl_id')['symbol'], on='ensembl_id')

# Rename column for clarity
adata.var.rename(columns={'symbol': 'gene_symbol'}, inplace=True)

# Optional : rescue highly expressed missing genes
rescue_mask = (adata.var['status'] == 'missing') & (adata.var['log1p_total_counts'] > 7.5)
adata.var.loc[rescue_mask, 'status'] = 'rescued'

In [ ]:
# Not for github, checking count data

import numpy as np
import scipy.sparse as sp

# 1️⃣ Convert sparse to dense (temporarily) for inspection
X = adata.X.toarray() if sp.issparse(adata.X) else adata.X

# 2️⃣ Basic stats
print("Min:", X.min(), "Max:", X.max(), "Mean:", X.mean())

# 3️⃣ Simple heuristic check
# Count matrices are integer (or very close to)
is_integer_like = np.allclose(X, np.round(X))
# Log-transformed matrices have smaller dynamic range (often 0–10) and non-integers
looks_log = X.max() < 50 and not is_integer_like

if is_integer_like:
    print("✅ Likely raw count data.")
elif looks_log:
    print("⚠️ Likely log-transformed data.")
else:
    print("❓ Ambiguous — check your preprocessing history.")

# 4️⃣ Optional sanity check on value distribution
import matplotlib.pyplot as plt
plt.hist(X.flatten(), bins=100, range=(0, 10))
plt.title("Value distribution of adata.X")
plt.xlabel("Expression value")
plt.ylabel("Frequency")
plt.show()
